# Dataset

In this section, the avaibale dataset used in this project are presented.

## NYS population distribution in counties

In [230]:
import pandas as pd
import numpy as np
url = 'https://wikipedia.org/wiki/List_of_counties_in_New_York'
df = pd.read_html(url)[2]
df.head()

,County,FIPS Code[3],County seat[5],Est.[5],Formed from[1],Named for[2],Density (Pop./mi2),Pop. (2010)[6],Area[5],Map
0,Albany County,1,Albany,1683,One of 12 original counties created in the New...,James II of England (James VII of Scotland) (1...,570.74,304204,"533 sq mi(1,380 km2)",NaN
1,Allegany County,3,Belmont,1806,Genesee County,A variant spelling of the Allegheny River,47.34,48946,"1,034 sq mi(2,678 km2)",NaN
2,Bronx County,5,none,1914[7],New York County,"Jonas Bronck (1600?–1643), an early settler of...",24118.20,1385108,57.43 sq mi(149 km2),NaN
3,Broome County,7,Binghamton,1806,Tioga County,"John Broome (1738–1810), fourth Lieutenant Gov...",280.56,200600,"715 sq mi(1,852 km2)",NaN
4,Cattaraugus County,9,Little Valley,1808,Genesee County,A word from an uncertain Iroquoian language me...,61.31,80317,"1,310 sq mi(3,393 km2)",NaN


### Remove outlier columns

In [231]:
df1 = pd.DataFrame()
df1 = df[["County","Pop. (2010)[6]","Density (Pop./mi2)"]].copy()
df1 = df1.rename(columns={"Pop. (2010)[6]": "Population"})
df1['Area (mi2)'] = df1['Population']/df1['Density (Pop./mi2)']
df1['Area (mi2)'] = df1['Area (mi2)'].round(2)
df1.head()

,County,Population,Density (Pop./mi2),Area (mi2)
0,Albany County,304204,570.74,533.00
1,Allegany County,48946,47.34,1033.92
2,Bronx County,1385108,24118.20,57.43
3,Broome County,200600,280.56,715.00
4,Cattaraugus County,80317,61.31,1310.01
...,...,...,...,...
57,Washington County,63216,74.72,846.04
58,Wayne County,93772,67.75,1384.09
59,Westchester County,949113,1898.23,500.00
60,Wyoming County,42155,70.73,596.00


## Location of each county

In [233]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

AddresLat = list()
AddresLong = list()


for county in df1['County']:
    address =  county + ', NY'
    
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    AddresLat.append(latitude)
    AddresLong.append(longitude)

In [329]:
df1['Latitude'] =  AddresLat
df1['Longitude'] =  AddresLong
df1.head()

,County,Population,Density (Pop./mi2),Area (mi2),Latitude,Longitude
0,Albany,304204,570.74,533.00,42.598690,-73.984400
1,Allegany,48946,47.34,1033.92,42.244606,-78.041928
2,Bronx,1385108,24118.20,57.43,40.850485,-73.840404
3,Broome,200600,280.56,715.00,42.145562,-75.840411
4,Cattaraugus,80317,61.31,1310.01,42.223482,-78.647710


## Average land price in each county

In [327]:
url = 'https://www.tax.ny.gov/research/property/assess/sales/resmedian.htm'
df = pd.read_html(url)[0]
df.head()

Unnamed: 0_level_0    2017              2018              2019          
              County # Sales    Median # Sales    Median # Sales    Median
0             Albany    3403  $207,000    3434  $210,000    3445  $217,500
1           Allegany     523   $61,500     509   $67,000     482   $75,500
2             Broome    1536  $110,000    1584  $116,000    1746  $110,000
3        Cattaraugus     804   $79,500     811   $80,000     843   $87,250
4             Cayuga     743  $117,000     781  $122,000     786  $120,250

### Remove outlier columns

In [328]:
CountyData = df.iloc[:, 0]
PriceData = df.iloc[:, 6]
df2 = pd.DataFrame()
df2['County'] =  CountyData
df2['LandPrice'] =  PriceData
df2['LandPrice'] = df2['LandPrice'].str.replace(',', '')
df2['LandPrice'] = df2['LandPrice'].str.replace('$', '')
df2['LandPrice'] = df2['LandPrice'].astype(int)
df2.head()

,County,LandPrice
0,Albany,217500
1,Allegany,75500
2,Broome,110000
3,Cattaraugus,87250
4,Cayuga,120250


## Merge two dataframes

In [279]:
df1['County'] = df1['County'].str.replace(' County', '')
dfinal = df1
dfinal = dfinal.merge(df2,how='left', left_on='County', right_on='County')
dfinal.head()

,County,Population,Density (Pop./mi2),Area (mi2),Latitude,Longitude,LandPrice
0,Albany,304204,570.74,533.00,42.598690,-73.984400,217500.0
1,Allegany,48946,47.34,1033.92,42.244606,-78.041928,75500.0
2,Bronx,1385108,24118.20,57.43,40.850485,-73.840404,NaN
3,Broome,200600,280.56,715.00,42.145562,-75.840411,110000.0
4,Cattaraugus,80317,61.31,1310.01,42.223482,-78.647710,87250.0


In [324]:
latitude = 43.015598
longitude = -76.225713
Countis_map = folium.Map(location=[latitude, longitude], zoom_start=7) # generate map centred around the Conrad Hotel


# add the Italian restaurants as blue circle markers
for lat, lng, label, rad in zip(dfinal.Latitude, dfinal.Longitude, dfinal.County, dfinal.Population):
    folium.features.CircleMarker(
        [lat, lng],
        radius=rad/200000,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(Countis_map )

# display map
Countis_map 

## Add average prices to missed values for prices

In [336]:
missing_data = dfinal.isnull()
missing_data.head()
avg_price = dfinal.LandPrice.astype("float").mean(axis=0)
print("Average of price:", avg_price)
dfinal.LandPrice.replace(np.nan, avg_price, inplace=True)

Average of prices: 179258.07142857142


In [339]:
latitude = 43.015598
longitude = -76.225713
Countis_map = folium.Map(location=[latitude, longitude], zoom_start=7) # generate map centred around the Conrad Hotel

# add the Italian restaurants as blue circle markers
for lat, lng, label, rad in zip(dfinal.Latitude, dfinal.Longitude, dfinal.County, dfinal.LandPrice):
    folium.features.CircleMarker(
        [lat, lng],
        radius=rad/20000,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(Countis_map )

# display map
Countis_map 

To see the map use this link!  
https://nbviewer.jupyter.org/github/haamedmomeni/github-example/blob/master/Data.ipynb